# Проект по SQL

Моя первая задача как аналитика проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

# Описание данных 

**ER-диаграмма**

![ER-диаграмма](https://ltdfoto.ru/images/2024/10/20/SNIMOK-EKRANA-2024-10-20-220130.png)

**Таблица** books  
Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.  

**Таблица** authors  
Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.  

**Таблица** publishers  
Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;  

**Таблица** ratings  
Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.  

**Таблица** reviews  
Содержит данные о пользовательских обзорах на книги:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя пользователя, написавшего обзор;
- text — текст обзора.

# Выполнение заданий

In [1]:
import pandas as pd
import sqlalchemy as sa

In [2]:
db_config = {
    'user': 'praktikum_student', # имя пользователя
    'pwd': 'Sdf4$2;d-d30pp', # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432, # порт подключения
    'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

## Исследуйте таблицы — выведите первые строки, посчитайте количество строк в каждой таблице.

In [3]:
def display_tables(*tables):
    for table in tables:
        query = f'''SELECT * 
                 FROM {table} 
                 LIMIT 5'''
        query_count_rows = f'''SELECT COUNT(*) 
                             FROM {table}'''
        print(f'\033[1mТаблица {table}\033[0m')
        print('Количество строк в таблице: {}'.format(get_sql_data(query_count_rows).loc[0, 'count']))
        display(get_sql_data(query))

In [4]:
display_tables('books', 'authors', 'publishers', 'ratings', 'reviews')

Таблица books
Количество строк в таблице: 1000


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица authors
Количество строк в таблице: 636


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица publishers
Количество строк в таблице: 340


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблица ratings
Количество строк в таблице: 6456


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица reviews
Количество строк в таблице: 2793


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Вывод:** каждая таблица соответствуют описанию и обозначенной структуре.

## Посчитайте, сколько книг вышло после 1 января 2000 года

In [5]:
get_sql_data('''SELECT COUNT(*) \
             FROM books \
             WHERE publication_date > '2000-01-01'
             ''')

,count
0,819


**Вывод:** 819 книг вышло после 1 января 2000 года.

## Для каждой книги посчитайте количество обзоров и среднюю оценку

In [6]:
get_sql_data('''WITH amount_reviews AS (SELECT book_id,
                                               COUNT(review_id) AS amount_reviews
                                        FROM reviews
                                        GROUP BY book_id),
                     average_rating AS (SELECT book_id,
                                               ROUND(AVG(rating), 2) AS average_rating
                                        FROM ratings
                                        GROUP BY book_id)
                SELECT b.book_id,
                       title,
                       amount_reviews,
                       average_rating
                FROM books AS b
                LEFT JOIN amount_reviews AS am_r ON b.book_id = am_r.book_id
                LEFT JOIN average_rating AS av_r ON b.book_id = av_r.book_id''')

,book_id,title,amount_reviews,average_rating
0,652,The Body in the Library (Miss Marple #3),2.0,4.50
1,273,Galápagos,2.0,4.50
2,51,A Tree Grows in Brooklyn,5.0,4.25
3,951,Undaunted Courage: The Pioneering First Missio...,2.0,4.00
4,839,The Prophet,4.0,4.29
...,...,...,...,...
995,64,Alice in Wonderland,4.0,4.23
996,55,A Woman of Substance (Emma Harte Saga #1),2.0,5.00
997,148,Christine,3.0,3.43
998,790,The Magicians' Guild (Black Magician Trilogy #1),2.0,3.50


**Вывод:** для каждой книги нашел количество обзоров и среднюю оценку.

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц

In [7]:
get_sql_data('''WITH popular_publisher AS (SELECT publisher_id,
                                                   COUNT(book_id) AS amount_of_books
                                            FROM books 
                                            WHERE num_pages > 50 
                                            GROUP BY publisher_id 
                                            ORDER BY COUNT(book_id) DESC 
                                            LIMIT 1)
                 SELECT pub.publisher_id,
                        publisher,
                        amount_of_books
                 FROM publishers AS pub
                 JOIN popular_publisher AS pop_pub ON pub.publisher_id = pop_pub.publisher_id
                 ''')
#В CTE формирую идентификатор издателя и количество выпущенных им книг

,publisher_id,publisher,amount_of_books
0,212,Penguin Books,42


**Вывод:** Penguin Books - издательство, которое выпустило наибольшее число книг толще 50 страниц

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [8]:
get_sql_data('''WITH best_author AS (SELECT author_id,
                                            ROUND(AVG(rating), 2) AS avg_rating
                                     FROM books AS b
                                     LEFT JOIN ratings AS r ON b.book_id = r.book_id 
                                     WHERE b.book_id IN (SELECT b.book_id 
                                                         FROM books AS b 
                                                         LEFT JOIN ratings AS r ON b.book_id = r.book_id 
                                                         GROUP BY b.book_id 
                                                         HAVING COUNT(rating) >= 50) 
                                     GROUP BY author_id 
                                     ORDER BY AVG(rating) DESC 
                                     LIMIT 1)
                 SELECT a.author_id,
                        author,
                        avg_rating
                 FROM authors AS a 
                 JOIN best_author AS ba ON a.author_id = ba.author_id''')
        
#В первом подзапросе формирую идентификаторы книг по условию. В CTE id автора с самой высокой средней оценкой книг

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.29


**Вывод:** J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой книг.

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [9]:
get_sql_data('''SELECT AVG(number_of_reviews)
              FROM (SELECT COUNT(review_id) AS number_of_reviews
                    FROM books AS b
                    JOIN reviews AS re ON b.book_id = re.book_id
                    WHERE username IN (SELECT username
                                          FROM ratings
                                          GROUP BY username
                                          HAVING COUNT(rating) > 48)
                    GROUP BY username) AS table_number_of_reviews''')
#В первом подзапросе формирую пользователей, которые поставили больше 48 оценок;
#Во втором подзапросе вывожу столбец с числом обзоров в разрезе по пользователям из первого запроса

,avg
0,24.0


**Вывод:**  Пользователи, которые поставили больше 48 оценок, сделали в среднем 24 обзора.

# Общий вывод

1. Каждая таблица соответствует заданному описанию и обозначенной структуре. После 1 января 2000 года было выпущено 819 книг. Для каждой книги было определено количество обзоров и средняя оценка. Издательство Penguin Books выпустило наибольшее количество книг, а именно 42, объемом более 50 страниц. J.K. Rowling/Mary GrandPré стала автором с самой высокой средней оценкой книг — 4,29. Пользователи, которые поставили более 48 оценок, в среднем написали 24 обзора.
2. Так как пользователи, которые ставят более 48 оценок, в среднем пишут 24 обзора, это указывает на высокую вовлеченность.  Рекомендуется:
- Разработать программу лояльности для активных рецензентов, например, предоставление им доступа к эксклюзивным материалам или предстоящим книгам для обзора.
- Проанализировать, какие темы и жанры наиболее популярны среди тех, кто оставляет большое количество отзывов, и использовать эти данные для разработки новых произведений.